In [1]:
import pandas as pd
import numpy as np 
import uuid

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
df0 = pd.read_excel('asset-v1_SkillFactory+MFTIDS+SEP2023+type@asset+block@customer_and_transaction__2_.xlsx', sheet_name='transaction')
df_customer = pd.read_excel('asset-v1_SkillFactory+MFTIDS+SEP2023+type@asset+block@customer_and_transaction__2_.xlsx', sheet_name='customer')

In [4]:
df_customer.loc[df_customer['gender'].isin(['F', 'Femal']), 'gender'] = 'Female'
df_customer.loc[df_customer['gender'] == 'M', 'gender'] = 'Male'

In [5]:
df_customer['address_code'] = df_customer.address.apply(lambda x: x.split(' ')[0])
df_customer['address_name'] = df_customer.address.apply(lambda x: ' '.join(x.split(' ')[1:]))

In [6]:
df_customer = df_customer.drop(columns='address')

In [7]:
df_product = df0[['product_id','brand','product_line','product_class','product_size']]

In [8]:
df_product_ = df_product.drop_duplicates() 

In [9]:
df_product_['product_uid'] = df_product_['product_id'].apply(lambda x: uuid.uuid4())

In [10]:
df0 = df0.merge(df_product_, how='left', on=['product_id','brand','product_line','product_class','product_size'])

In [11]:
df_transaction = df0[['transaction_id','product_uid','customer_id','transaction_date','online_order','order_status','list_price', 'standard_cost']]

In [13]:
df_product_ = df_product_.replace({np.nan: None})
df_transaction = df_transaction.replace({np.nan: None})
df_customer = df_customer.replace({np.nan: None})

In [14]:
import psycopg2 

import psycopg2.extras as extras 
import pandas as pd 


def execute_values(conn, df, table): 

	tuples = [tuple(x) for x in df.to_numpy()] 

	cols = ','.join(list(df.columns)) 
	# SQL query to execute 
	query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols) 
	cursor = conn.cursor() 
	try: 
		extras.execute_values(cursor, query, tuples) 
		conn.commit() 
	except (Exception, psycopg2.DatabaseError) as error: 
		print("Error: %s" % error) 
		conn.rollback() 
		cursor.close() 
		return 1
	print("the dataframe is inserted") 
	cursor.close() 


conn = psycopg2.connect( 
	database="postgres", user='postgres', password='admin', host='127.0.0.1', port='5432'
) 


In [15]:
# df_order['uid']=df_order['uid'].astype(str)
df_transaction['product_uid']=df_transaction['product_uid'].astype(str)
df_product_['product_uid']=df_product_['product_uid'].astype(str)

In [16]:
execute_values(conn, df_customer, 'module1.customer') 

the dataframe is inserted


In [17]:
execute_values(conn, df_product_, 'module1.product') 

the dataframe is inserted


In [20]:
customer_temp = pd.DataFrame(data={'customer_id':[5034], 'first_name':[None], 'last_name':[None], 'gender':[None], 'DOB':[None], 'job_title':[None],
       'job_industry_category':[None], 'wealth_segment':[None], 'deceased_indicator':[None], 'owns_car':[None],
        'address_code':[None], 'address_name':[None], 'postcode':[None], 'state':[None], 'country':[None], 'property_valuation':[None]})

In [21]:
#Error: ОШИБКА:  INSERT или UPDATE в таблице "order" нарушает ограничение внешнего ключа "order_customer_id_fkey"
#DETAIL:  Ключ (customer_id)=(5034) отсутствует в таблице "customer".
execute_values(conn, customer_temp, 'module1.customer') 

the dataframe is inserted


In [22]:
execute_values(conn, df_transaction, 'module1.transaction') 

the dataframe is inserted


In [ ]:
### ОПИСАНИЕ РЕШЕНИЯ ###

Выделим три таблицы: customer, product, transaction
Для преведения их к 3НФ сделаем следующие шаги:
1) все таблицы не содержат составных данных в ячейках/столбцах
2) таблица customer не содержит дубликатов, содержит уникальный первичный ключ customer_id, при этом в колонке gender таблицы customers есть неуникальные названия - переименуем. Так же разделим поле address на поля address_code и address_name
3) таблица product содержит дубликаты, product_id не является уникальным, уникальным является сочетание всех столбцов - удалим дубликаты и введем уникальный product_uid
4) таблица transaction не содержит дубликатов, содержит уникальный первичный ключ transaction_id, + добавляем product_uid, оставляем customer_id
5) таблица customer не содержит (customer_id)=(5034) поэтому добавим customer_id с пустыми полями в таблицу 

Теперь таблицы находятся в 3НФ: 1НФ - атрибуты простые, скалярные, нет дублей в записях, 2НФ - 1НФ + каждый не ключевой атрибут неприводимо зависит от первичного ключа (составе потенциального ключа отсутствует меньшее подмножество атрибутов), 3НФ - 2НФ +  каждый не ключевой атрибут нетранзитивно зависит от первичного ключа (все не ключевые поля, содержимое которых может относиться к нескольким записям таблицы вынесены в отдельные таблицы).

